In [4]:
import os
from pathlib import Path
import pandas as pd

from sklearn.datasets import load_diabetes

from datahandles import preprocess_numeric
from utils import Config

from tabllm import load_train_validation_test

In [6]:
dataset = load_train_validation_test(dataset_name='car', data_dir=Path('./tabllm/data/datasets/car'))

/export/pasand/trans-hyper/tabllm/create_external_datasets.py:320: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset['label'] = dataset['label'].replace(label_dict)


In [20]:
dataset['train']['label'].value_counts()

label
0    1012
1     321
3      26
2      23
Name: count, dtype: int64

In [18]:
cfg = Config("./cfgs/base.yaml")
data = load_diabetes(as_frame=True)
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target
df["target"] = (df["target"] > df["target"].median()).astype(int)
df.head()
# df = data.frame.copy()
# df["target"] = (df["target"] > df["target"].median()).astype(int)  # convert to binary
# data = preprocess_numeric(cfg, df, target_col="target", n_features=cfg.datasets.adult.n_features())

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646,1
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204,0
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930,1
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362,1
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641,0


In [23]:
print(df["target"].value_counts())

target
1    221
0    221
Name: count, dtype: int64
